In [1]:
#missionaries and cannibals problem is a famous toy problem in artificial intelligence
#details of the game could be found in the following link
# https://en.wikipedia.org/wiki/Missionaries_and_cannibals_problem
#we would try a bunch of different algorithms including bfs,dfs and dijkstra to get the solution
#plz note that this answer is not the only solution

#details of graph adt can be found in the following link
# https://github.com/je-suis-tm/graph-theory/blob/master/BFS%20DFS%20on%20DCG.ipynb
class graph:
        def __init__(self):
            self.graph={}
            self.visited={}
    
        def append(self,vertexid,edge,weight):
            if vertexid not in self.graph.keys():          
                self.graph[vertexid]={}
                self.visited[vertexid]=0
            self.graph[vertexid][edge]=weight
            
        def reveal(self):
            return self.graph
        
        def vertex(self):
            return list(self.graph.keys())
    
        def edge(self,vertexid):
            return list(self.graph[vertexid].keys())
        
        def weight(self,vertexid,edge):
            return (self.graph[vertexid][edge])
        
        def size(self):
            return len(self.graph)
        
        def visit(self,vertexid):
            self.visited[vertexid]=1
            
        def go(self,vertexid):
            return self.visited[vertexid]
        
        def route(self):
            return self.visited

In [2]:
#details of dijkstra algo could be found in the following link
# https://github.com/je-suis-tm/graph-theory/blob/master/dijkstra%20shortest%20path.ipynb
def dijkstra(df,start,end):
    queue={}
    distance={}
    queue[start]=0
    pred={}

    for i in df.vertex():
        distance[i]=float('inf')
    distance[start]=0    
        
    while queue:
        temp=min(queue,key=queue.get)
        queue.pop(temp)
        for j in df.edge(temp):
            if distance[temp]+df.weight(temp,j)<distance[j]:
                distance[j]=distance[temp]+df.weight(temp,j)
                pred[j]=temp
                
            if df.go(j)==0 and j not in queue:
                queue[j]=distance[j]
            
        df.visit(temp)
        if temp==end:
            break
    
    k=end
    path=[]
    while pred:
        path.insert(0,k)
        if k==start:
            break
        k=pred[k]
        
    return distance[end],path

#details of bfs and dfs can be found in the following link
# https://github.com/je-suis-tm/graph-theory/blob/master/BFS%20DFS%20on%20DCG.ipynb
def bfs(df,start,end):
    queue=[]
    queue.append(start)
    pred={}
    c=0
    
    while queue:
        temp=queue.pop(0)
        df.visit(temp)
        for newpos in df.edge(temp):
            if df.go(newpos)==0 and newpos not in queue:
                queue.append(newpos)
                pred[newpos]=temp
                
        if temp==end:
            break
        
        c+=1
        
    k=end
    path=[]
    while pred:
        path.insert(0,k)
        if k==start:
            break
        k=pred[k]
        
    return len(path)-1,path


#
def dfs_itr(df,start,end):
    queue=[]
    queue.append(start)
    pred={}
    c=0
    
    while queue:
        temp=queue.pop(0)
        smallq=[]
        df.visit(temp)
        for newpos in df.edge(temp):
            if df.go(newpos)==0:
                if newpos in queue:
                    queue.remove(newpos)
                smallq.append(newpos)
                pred[newpos]=temp
                
        queue=smallq+queue
        
        if temp==end:
            break
        
        c+=1
        
    k=end
    path=[]
    while pred:
        path.insert(0,k)
        if k==start:
            break
        k=pred[k]
        
    return len(path)-1,path

In [3]:
#the tricky part is to convert the possibilities into a graph structure
#denote a vertex (a,b,c,d,e)
#a is the number of cannibals on left bank
#b is the number of missionaries on left bank
#c is the number of cannibals on right bank
#d is the number of missionaries on right bank
#e is the location of a boat
#denote 0 as the boat is on left bank
#1 as the boat is on right bank
#what we intend to do is to build up all the possible occasions as vertices
#and convert the valid moves from one status to another into edges
#weights are 1 for the purpose of dijkstra

#validate function is to validate the status
#for either bank, number of missionaries cannot be smaller than number of cannibals
#and number of missionaries or cannibals for either bank cannot be negative
#the reason of negative numbers can be seen in In[5]
def validate(a,b,c,d):
    if a<0 or b<0 or c<0 or d<0:
        return False
    if (a>b and b!=0) or (c>d and d!=0):
        return False
    else:
        return True

In [4]:
df=graph()

In [5]:
#to build up vertices
#lets split all the possibilities in two
#when the boat is on left bank and on right bank
#when the boat is on left bank, all valid edges should be the boat is on right bank
#cuz we wanna ship people from left to right and ship cant move by itself
#vice versa

#lets look at the occasion when when the boat is on left bank
#denote i as number of cannibals on left bank
#denote j as number of missionaries on left bank
#and 3-i is apparently the number of cannibals on right bank
#cuz the sum of cannibals is 3
#the same idea applies to missionaries
#this way would save us a lot of time from iterations
#and the valid edges should be the boat from left bank to right bank
#which is 0 to 1

#the boat can only carry two people
#there are five possible occasions for crossing river
#only one person on boat, carry one cannibal, carry one missionary
#two people on boat, carry two cannibals, carry two missionaries, carry one cannibal and one missionary
#for instance, the maths representation of carry two cannibals is
#from (i,j,3-i,3-j,0) to (i-2,j,3-i+2,3-j,1)
#two cannibals are extracted from left bank and added to the right bank
#missionaries for both banks stay the same
#the boat changes from 0 to 1

#we use validate function to ensure that cannibals would not outnumber missionaries
#after that we add edges to the graph for when the boat is on left bank
for i in range(0,4):
    for j in range(0,4):
        if validate(i-2,j,3-i+2,3-j):
            df.append((i,j,3-i,3-j,0),(i-2,j,3-i+2,3-j,1),1)
        if validate(i,j-2,3-i,3-j+2):
            df.append((i,j,3-i,3-j,0),(i,j-2,3-i,3-j+2,1),1)
        if validate(i-1,j-1,3-i+1,3-j+1):
            df.append((i,j,3-i,3-j,0),(i-1,j-1,3-i+1,3-j+1,1),1)
        if validate(i-1,j,3-i+1,3-j):
            df.append((i,j,3-i,3-j,0),(i-1,j,3-i+1,3-j,1),1)
        if validate(i,j-1,3-i,3-j+1):
            df.append((i,j,3-i,3-j,0),(i,j-1,3-i,3-j+1,1),1)
            
#vice versa
for i in range(0,4):
    for j in range(0,4):
        if validate(i+1,j,3-i-1,3-j):
            df.append((i,j,3-i,3-j,1),(i+1,j,3-i-1,3-j,0),1)
        if validate(i,j+1,3-i,3-j-1):
            df.append((i,j,3-i,3-j,1),(i,j+1,3-i,3-j-1,0),1)
        if validate(i+2,j,3-i-2,3-j):
            df.append((i,j,3-i,3-j,1),(i+2,j,3-i-2,3-j,0),1)
        if validate(i,j+2,3-i,3-j-2):
            df.append((i,j,3-i,3-j,1),(i,j+2,3-i,3-j-2,0),1)
        if validate(i+1,j+1,3-i-1,3-j-1):
            df.append((i,j,3-i,3-j,1),(i+1,j+1,3-i-1,3-j-1,0),1)

In [6]:
#we use deep copy to create copies of graph adt for other algo tests
import copy
df1=copy.deepcopy(df)
df2=copy.deepcopy(df)


In [7]:
import datetime as dt

t1=dt.datetime.now()
print(dijkstra(df,(3,3,0,0,0),(0,0,3,3,1)))
t2=dt.datetime.now()
print('dijkstra:',(t2-t1).microseconds)

(11, [(3, 3, 0, 0, 0), (1, 3, 2, 0, 1), (2, 3, 1, 0, 0), (0, 3, 3, 0, 1), (1, 3, 2, 0, 0), (1, 1, 2, 2, 1), (2, 2, 1, 1, 0), (2, 0, 1, 3, 1), (3, 0, 0, 3, 0), (1, 0, 2, 3, 1), (2, 0, 1, 3, 0), (0, 0, 3, 3, 1)])
dijkstra: 0


In [8]:
t1=dt.datetime.now()
print(bfs(df1,(3,3,0,0,0),(0,0,3,3,1)))
t2=dt.datetime.now()
print('bfs:',(t2-t1).microseconds)

(11, [(3, 3, 0, 0, 0), (1, 3, 2, 0, 1), (2, 3, 1, 0, 0), (0, 3, 3, 0, 1), (1, 3, 2, 0, 0), (1, 1, 2, 2, 1), (2, 2, 1, 1, 0), (2, 0, 1, 3, 1), (3, 0, 0, 3, 0), (1, 0, 2, 3, 1), (2, 0, 1, 3, 0), (0, 0, 3, 3, 1)])
bfs: 0


In [9]:
t1=dt.datetime.now()
print(dfs_itr(df2,(3,3,0,0,0),(0,0,3,3,1)))
t2=dt.datetime.now()
print('dfs:',(t2-t1).microseconds)

(11, [(3, 3, 0, 0, 0), (1, 3, 2, 0, 1), (2, 3, 1, 0, 0), (0, 3, 3, 0, 1), (1, 3, 2, 0, 0), (1, 1, 2, 2, 1), (2, 2, 1, 1, 0), (2, 0, 1, 3, 1), (3, 0, 0, 3, 0), (1, 0, 2, 3, 1), (2, 0, 1, 3, 0), (0, 0, 3, 3, 1)])
dfs: 0
